### A simple training demo

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from netam.framework import (
    SHMoofDataset,
    RSSHMBurrito,
)
from epam.torch_common import pick_device

from shmex.shm_data import load_shmoof_dataframes, dataset_dict
from shmex import shm_zoo

Using Metal Performance Shaders


We're just going to use shmoof training data, the same data used to train the context NT model.

In [2]:
site_count = 500
train_df, val_df = load_shmoof_dataframes(dataset_dict["shmoof"], val_nickname="small")
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
subsampled_train_df = train_df.iloc[::int(len(train_df)/5000)].copy().reset_index(drop=True)

In [3]:
device = pick_device()
train_data_5mer = SHMoofDataset(train_df, kmer_length=5, site_count=site_count)
val_data_5mer = SHMoofDataset(val_df, kmer_length=5, site_count=site_count)

train_data_3mer = SHMoofDataset(train_df, kmer_length=3, site_count=site_count)
val_data_3mer = SHMoofDataset(val_df, kmer_length=3, site_count=site_count)

for data in [train_data_5mer, val_data_5mer, train_data_3mer, val_data_3mer]:
    data.to(device)

Using Metal Performance Shaders


In [8]:
model_name = "cnn_joi_sml"
training_method = "full"
burrito_name = f"{model_name}_{training_method}_100"
cnn_model = shm_zoo.create_model(model_name)
cnn_model.to(device)
cnn_burrito = RSSHMBurrito(train_data_3mer, val_data_3mer, cnn_model, name=burrito_name)
cnn_burrito.joint_train(training_method=training_method, epochs=100, cycle_count=5)
cnn_burrito.save_crepe(f"trained_models/{burrito_name}")
cnn_crepe = cnn_burrito.to_crepe()

Epoch:  66%|██████▌   | 66/100 [17:53<09:12, 16.26s/it, loss_diff=1.989e-06, lr=9.35e-5, val_loss=0.06529]  
